In [1]:

import os, re, glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ---------------------------
# Config
# ---------------------------
input_dir  = "figs/projections/chest/projections"
output_dir = "figs/projections/chest/mask_edge"
os.makedirs(output_dir, exist_ok=True)

# 掩码模式：
#   'edge' 使用边缘检测；'threshold' 区间阈值；'extreme' 屏蔽极值(0/255)
MASK_MODE   = "edge"

# 当使用 'threshold'/'extreme' 时的参数
LOW_RATIO   = 0.00
HIGH_RATIO  = 0.95

# 当使用 'edge' 时的参数
EDGE_METHOD   = "sobel"   # 'sobel' or 'canny'

# sobel 的可选参数
EDGE_SIGMA    = 1.2       # 仅作预平滑用（Sobel 时生效；Canny 自有 σ）
EDGE_PCT      = 50.0      # Sobel 的百分位阈值

# canny 的可选参数
CANNY_SIGMA   = [0.8, 1.2, 1.8]  # 多尺度；也可用单个 float，如 1.2
CANNY_LOW     = "auto"    # 'auto' 或 [0,1] 浮点数
CANNY_HIGH    = "auto"    # 'auto' 或 [0,1] 浮点数

USE_CLAHE     = True      # 是否对比度增强（更容易抓到弱边）
CLAHE_CLIP    = 0.01      # CLAHE clip limit
REMOVE_SMALL  = True      # 去除小噪点
MIN_SIZE_PX   = 16        # 小连通域像素阈值（视分辨率再调）

print("Config loaded.")
print("input_dir:", input_dir)
print("output_dir:", output_dir)


Config loaded.
input_dir: figs/projections/chest/projections
output_dir: figs/projections/chest/mask_edge


In [2]:
from skimage.feature import canny
from skimage.filters import sobel, gaussian
from skimage import exposure, morphology, io, util

def to_uint8_gray(img):
    """统一转灰度 uint8"""
    arr = img
    if arr.ndim == 3 and arr.shape[2] == 4:  # RGBA -> RGB
        arr = arr[:, :, :3]
    if arr.ndim == 3 and arr.shape[2] == 3:
        if np.issubdtype(arr.dtype, np.floating):
            gray = arr.mean(axis=2)           # [0,1]
            gray = np.clip(gray * 255.0, 0, 255).astype(np.uint8)
        else:
            gray = arr.mean(axis=2).round().astype(np.uint8)
    elif arr.ndim == 2:
        if np.issubdtype(arr.dtype, np.floating):
            gray = np.clip(arr * 255.0, 0, 255).astype(np.uint8)
        else:
            gray = arr.astype(np.uint8)
    else:
        raise ValueError(f"Unsupported image shape: {arr.shape}")
    return gray

def _auto_canny_thresholds(img01: np.ndarray, low_pct=60, high_pct=90):
    """基于百分位自适应推断 Canny low/high 阈值（输入已归一化 0~1）。"""
    g = gaussian(img01, sigma=0.8, preserve_range=True)
    low = np.percentile(g, low_pct)
    high = np.percentile(g, high_pct)
    eps = 1e-6
    if not np.isfinite(low) or not np.isfinite(high) or high <= low + eps:
        low, high = 0.1, 0.3
    return float(np.clip(low, 0.0, 1.0)), float(np.clip(high, 0.0, 1.0))

def build_edge_mask(
    gray: np.ndarray,
    method: str = EDGE_METHOD,              # 'sobel' or 'canny'
    gaussian_sigma: float = EDGE_SIGMA,     # 仅 sobel 预平滑
    thresh_mode: str = "percentile",
    thresh_value: float = EDGE_PCT,
    canny_sigma = CANNY_SIGMA,              # float 或 list[float]
    canny_low = CANNY_LOW,                  # 'auto' 或 float
    canny_high = CANNY_HIGH,                # 'auto' 或 float
    use_clahe: bool = USE_CLAHE,
    clahe_clip: float = CLAHE_CLIP,
    remove_small: bool = REMOVE_SMALL,
    min_size_px: int = MIN_SIZE_PX
) -> np.ndarray:
    """
    生成二值 mask，返回 uint8(0/1)：0=边缘，1=非边缘（与原 pipeline 语义一致）。
    """
    g = gray.astype(np.float32) / 255.0

    # 局部对比度增强：有助于弱边显现
    if use_clahe:
        g = exposure.equalize_adapthist(g, clip_limit=clahe_clip).astype(np.float32)

    if method.lower() == "sobel":
        g_s = gaussian(g, sigma=gaussian_sigma, preserve_range=True) if gaussian_sigma and gaussian_sigma > 0 else g
        edge_mag = sobel(g_s)
        if thresh_mode == "percentile":
            t = np.percentile(edge_mag, thresh_value)
        elif thresh_mode == "value":
            t = float(thresh_value)
        else:
            raise ValueError("thresh_mode must be 'percentile' or 'value'.")
        edges = (edge_mag >= t)

    elif method.lower() == "canny":
        # 阈值：'auto' → 百分位；否则使用显式数字
        if isinstance(canny_low, str) or isinstance(canny_high, str):
            low_t, high_t = _auto_canny_thresholds(g, low_pct=60, high_pct=90)
        else:
            low_t, high_t = float(canny_low), float(canny_high)

        # 多尺度 σ：合并不同尺度的边（并集）
        sigmas = canny_sigma if isinstance(canny_sigma, (list, tuple)) else [float(canny_sigma)]
        edges_union = None
        for s in sigmas:
            e = canny(g, sigma=float(s), low_threshold=low_t, high_threshold=high_t)
            edges_union = e if edges_union is None else (edges_union | e)
        edges = edges_union

        # 去除小连通域噪声
        if remove_small:
            edges = morphology.remove_small_objects(edges, min_size=min_size_px)

    else:
        raise ValueError("method must be 'sobel' or 'canny'.")

    # 输出：边缘=0，其余=1
    return (~edges).astype(np.uint8)

def build_initial_mask(gray, mode="threshold", low_ratio=LOW_RATIO, high_ratio=HIGH_RATIO):
    """
    基础掩码：
    - mode="threshold": 仅保留 [low_ratio, high_ratio] 的像素为 1（其余 0）
    - mode="extreme":   将灰度==0 或 255 的像素置 0，其余 1
    - mode="edge":      使用边缘检测（边缘=0，其余=1）
    """
    if mode == "threshold":
        low  = int(255 * low_ratio)
        high = int(255 * high_ratio)
        mask = np.where((gray >= low) & (gray <= high), 1, 0).astype(np.uint8)
        return mask
    elif mode == "extreme":
        mask = np.where((gray == 0) | (gray == 255), 0, 1).astype(np.uint8)
        return mask
    elif mode == "edge":
        return build_edge_mask(gray)
    else:
        raise ValueError("mode must be 'threshold' or 'extreme' or 'edge'")

def angle_from_filename(path):
    m = re.search(r"deg_(\d+)\.png$", os.path.basename(path))
    if not m:
        raise ValueError(f"Bad filename: {path}")
    return int(m.group(1)) % 360


In [3]:

paths = glob.glob(os.path.join(input_dir, "deg_*.png"))
by_angle = {}
for p in paths:
    try:
        ang = angle_from_filename(p)
        by_angle[ang] = p
    except Exception as e:
        print("Skip:", p, "->", e)

mask_rates = []
visited = set()

for ang in sorted(by_angle.keys()):
    if ang in visited:
        continue
    mate = (ang + 180) % 360
    if mate not in by_angle:
        print(f"Skip angle {ang}: pair {mate} not found.")
        continue

    # 读取两张图
    img_a = plt.imread(by_angle[ang])
    img_b = plt.imread(by_angle[mate])
    gray_a = to_uint8_gray(img_a)
    gray_b = to_uint8_gray(img_b)

    # 生成初始 mask（根据 MASK_MODE 自动选择阈值/极值/边缘）
    mask_a0 = build_initial_mask(gray_a, mode=MASK_MODE, low_ratio=LOW_RATIO, high_ratio=HIGH_RATIO)
    mask_b0 = build_initial_mask(gray_b, mode=MASK_MODE, low_ratio=LOW_RATIO, high_ratio=HIGH_RATIO)

    # +180° 的初始 mask 水平翻转，对齐到 θ 的坐标系
    mask_b0_aligned = np.fliplr(mask_b0)

    # 并集（同一有效像素区域）
    union_mask_theta = (mask_a0 | mask_b0_aligned).astype(np.uint8)

    # 保存：θ 用 θ 坐标的并集；θ+180° 用“翻回去”的并集（保持各自原生坐标）
    out_a = os.path.join(output_dir,
        os.path.splitext(os.path.basename(by_angle[ang]))[0] + ".csv")
    out_b = os.path.join(output_dir,
        os.path.splitext(os.path.basename(by_angle[mate]))[0] + ".csv")

    pd.DataFrame(union_mask_theta).to_csv(out_a, index=False, header=False)
    pd.DataFrame(np.fliplr(union_mask_theta)).to_csv(out_b, index=False, header=False)

    kept = int(union_mask_theta.sum())
    total = union_mask_theta.size
    print(f"[{ang:03d} & {mate:03d}] kept {kept}/{total} ({kept/total:.2%}) -> {out_a}, {out_b}")

    mask_rates.append({
        "angle": ang,
        "mate": mate,
        "kept": kept,
        "total": total,
        "mask_rate": kept / total
    })

    visited.add(ang)
    visited.add(mate)

df_rates = pd.DataFrame(mask_rates)
out_summary = os.path.join(output_dir, "mask_rates_summary.csv")
df_rates.to_csv(out_summary, index=False)

avg_rate = df_rates["mask_rate"].mean() if len(df_rates) else float("nan")

print("✅ Pairwise-union masks (with +180° flip alignment) generated for all available angle pairs, using multi-scale Canny if configured.")
print(f"\n📊 average mask rate: {1-avg_rate:.2%} (save to {out_summary})" if len(df_rates) else "No pairs processed.")


[000 & 180] kept 42796/65536 (65.30%) -> figs/projections/chest/mask_edge/deg_00.csv, figs/projections/chest/mask_edge/deg_180.csv
[003 & 183] kept 43699/65536 (66.68%) -> figs/projections/chest/mask_edge/deg_03.csv, figs/projections/chest/mask_edge/deg_183.csv
[006 & 186] kept 44039/65536 (67.20%) -> figs/projections/chest/mask_edge/deg_06.csv, figs/projections/chest/mask_edge/deg_186.csv
[009 & 189] kept 44360/65536 (67.69%) -> figs/projections/chest/mask_edge/deg_09.csv, figs/projections/chest/mask_edge/deg_189.csv
[012 & 192] kept 45084/65536 (68.79%) -> figs/projections/chest/mask_edge/deg_12.csv, figs/projections/chest/mask_edge/deg_192.csv
[015 & 195] kept 45613/65536 (69.60%) -> figs/projections/chest/mask_edge/deg_15.csv, figs/projections/chest/mask_edge/deg_195.csv
[018 & 198] kept 45925/65536 (70.08%) -> figs/projections/chest/mask_edge/deg_18.csv, figs/projections/chest/mask_edge/deg_198.csv
[021 & 201] kept 46265/65536 (70.59%) -> figs/projections/chest/mask_edge/deg_21.cs